# 1) Preparación previa

### <u>Carga de librerías</u>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import unidecode

### <u>Carga de datasets</u>

In [ ]:
pbar = tqdm(["2015", "2016", "2017", "2018", "2019", "2020"])

for año in pbar:
    pbar.set_description(f"Procesando el año {año}. Datasets procesados")
    # Insertar la ubicación y el nombre de los .csv, quitando el año que se indica en el for loop con la variable "año"
    temp_location = "../Datasets/recorridos-realizados-" + año + ".csv"
    # Se usa un condicional para hacer un rename en el dataset 2020 que tiene unas columnas con otros nombres y se hace un drop de las columnas no utilizadas
    if año == "2020":
        temp_dataset = pd.read_csv(temp_location, low_memory = False).rename(columns={'periodo': 'año'})
        temp_dataset.drop(["fecha_origen_recorrido", "fecha_destino_recorrido", 'direccion_estacion_destino', 'direccion_estacion_origen'], axis = 1, inplace = True)
    else:
        temp_dataset = pd.read_csv(temp_location, low_memory = False).rename(columns={'periodo': 'año'})
        temp_dataset.drop(["fecha_origen_recorrido", "fecha_destino_recorrido", 'domicilio_estacion_destino', 'domicilio_estacion_origen'], axis = 1, inplace = True)
    temp_name = "dataset_" + año
    globals()[f"{temp_name}"] = temp_dataset

print("\nFin del proceso\n\nLas variables de cada dataset se llaman:\ndataset_2015\ndataset_2016\n...\n")

In [ ]:
### Un error lleva a que la carga normal no funcione en el dataset del 2021: pandas infiere la cantidad de columnas en base a la primera fila, lo cual no concuerda con la estructura de los datos.
# Para resolverlo, se le indican las columnas a pandas con un listado
col_names = ["ID", "Estado cerrado", "duracion_recorrido", "id_estacion_origen", "fecha_origen_recorrido", "nombre_estacion_origen", "fecha_destino_recorrido", "id_estacion_destino", "nombre_estacion_destino", "id_usuario", "Tipo de ciclista", "nombre", "Apellido de ciclista", "Msnbc de bicicleta", "Moto identificador público", "Código QR de bicicleta", "Modelo de bicicleta", "ID de factura", "ID de línea de factura", "Correo de ciclista", "Teléfono de ciclista", "ID de producto", "Origen de viaje", "Nombre de producto"]

dataset_2021 = pd.read_csv("../Datasets/recorridos-realizados-2021.csv", sep=",", names = col_names, low_memory = False, encoding='UTF-16 LE')

# Se eliminan las columnas indeseadas
dataset_2021.drop(["ID", "Estado cerrado", "fecha_origen_recorrido", "fecha_destino_recorrido", "Tipo de ciclista", "Apellido de ciclista", "Msnbc de bicicleta", "Moto identificador público", "Código QR de bicicleta", "Modelo de bicicleta", "ID de factura", "ID de línea de factura", "Correo de ciclista", "Teléfono de ciclista", "ID de producto", "Origen de viaje", "Nombre de producto"], axis = 1, inplace = True)

# Se procede a eliminar la fila con los títulos de columnas originales
dataset_2021.drop([0], axis = 0, inplace = True)

# Se agrega el período, que no estaba en el dataset original
dataset_2021["año"] = 2021

dataset_2021

# 2) Limpieza

### <u>Limpieza N° 1 - Género</u>

Los datasets del 2020 y del 2021 no incluyen esta columna y el dataset del 2019 tiene mayoritariamente nulos, por lo que se obtendrán de distintas formas:
    
- 1) el dataset 2021 contiene el nombre de los ciclistas, por lo que se puede hacer un merge con una base de datos de nombres y géneros. Los faltantes se imputaran respetando las proporciones de género preexistentes
- 2) una vez completado el susodicho dataset, se aprovechará que los 3 datasets contienen una columna de id_usuario para encontrar en el 2020 y en el 2019 los usuarios cuyo género esté en el 2021. Los faltantes se imputarán de la misma manera que en el anterior caso.

#### <u>2021:</u>

A) Carga del df de nombres y géneros:

In [ ]:
nombres = pd.read_csv("https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Gobierno%20Abierto%20(Python)/Datasets/nombres.csv", delimiter = "\t", encoding= "latin")
nombres.columns = ["nombre", "genero_usuario"]
nombres

B) Limpieza de la columna "nombre" en el dataset para aplicar merge:

In [ ]:
dataset_2021["nombre"] = dataset_2021["nombre"].str.lower()

# Se divide el string para separar casos con primer y segundo nombre:
dataset_2021["nombre"] = dataset_2021["nombre"].str.split()
# Se obtiene solo el primer nombre de la lista generada en la anterior línea de código:
dataset_2021.loc[:,"nombre"] = dataset_2021["nombre"].map(lambda x: x[0])
# Se limpia el formato de caracteres especiales:
dataset_2021["nombre"] = dataset_2021["nombre"].apply(lambda x : unidecode.unidecode(x))
# Se aplica el merge:
dataset_2021 = dataset_2021.merge(nombres, on ='nombre', how ='left')

dataset_2021

C) Aplicación del merge para imputar nulos:

In [ ]:
# Cálculo de resultados:
print("\nQuedaron:", dataset_2021.genero_usuario.isnull().sum(), "nulos restantes\nLa imputación anterior completó correctamente el", round(((len(dataset_2021)-dataset_2021.genero_usuario.isnull().sum())/len(dataset_2021)), 2)*100, "% de los datos faltantes\n")

D) Imputación de los últimos nulos:

In [ ]:
# Cálculo de las proporciones a utilizar
value_counts_2021 = round(dataset_2021.genero_usuario.value_counts(normalize = True),3)
value_counts_2021

In [ ]:
# Imputación:
dataset_2021.loc[dataset_2021["genero_usuario"].isnull(), "genero_usuario"] = np.random.choice(size=dataset_2021["genero_usuario"].isnull().sum(), a=["M", "F"], p=[value_counts_2021[0], value_counts_2021[1]])
# Se chequea que las proporciones se hayan mantenido similares:
round(dataset_2021.genero_usuario.value_counts(normalize = True),3)

In [ ]:
# Se corrobora que haya funcionado:
print("\nCantidad de datos nulos:", dataset_2021.genero_usuario.isnull().sum(), "\n")

In [ ]:
dataset_2021.isnull().sum()

In [ ]:
# Se puede eliminar la columna "nombre" que ya no será utilizada
dataset_2021.drop(["nombre"], axis = 1, inplace = True)

#### <u>2020:</u>

A) Creación de dataframe con único id_usuario del 2021:

Este paso es necesario pues muchos usuarios hicieron más de un recorrido. Al aparecer en más de una fila, un merge los duplicaría en el 2020

In [ ]:
# Se eliminan los recorridos con id_usuario duplicado:
dataframe_id_genero = dataset_2021.loc[:, ['id_usuario','genero_usuario']].drop_duplicates(subset = "id_usuario")
# Se transforma el dtype a integer para el merge:
id_int = dataframe_id_genero.id_usuario.astype("int64")
# Se reemplaza la serie object por la nueva serie integer:
dataframe_id_genero.id_usuario = id_int
# Se filtra solo los id_usuarios que se necesitan para 2020, pues un merge con usuarios de 2021 que no estén en 2020 agregaría filas:
dataframe_id_genero_2020 = dataframe_id_genero.loc[dataframe_id_genero.id_usuario.isin(dataset_2020.id_usuario), :]
dataframe_id_genero_2020

B) Aplicación del merge entre el nuevo dataframe y el dataset:

In [ ]:
dataset_2020 = pd.merge(dataset_2020, dataframe_id_genero_2020, how = "outer")
# Cálculo de resultados:
print("\nQuedaron:", dataset_2020.genero_usuario.isnull().sum(), "nulos restantes\nLa imputación anterior completó correctamente el", round(((len(dataset_2020)-dataset_2020.genero_usuario.isnull().sum())/len(dataset_2020)), 2)*100, "% de los datos faltantes\n")

C) Imputación de los últimos nulos:

In [ ]:
# Cálculo de las proporciones a utilizar
value_counts_2020 = round(dataset_2020["genero_usuario"].value_counts(normalize = True),3)
value_counts_2020

In [ ]:
# Imputación:
dataset_2020.loc[dataset_2020["genero_usuario"].isnull(), "genero_usuario"] = np.random.choice(size=dataset_2020["genero_usuario"].isnull().sum(), a=["M", "F"], p=[value_counts_2020[0], value_counts_2020[1]])
# Se chequea que las proporciones se hayan mantenido similares:
round(dataset_2020.genero_usuario.value_counts(normalize = True),3)

In [ ]:
# Se corrobora que haya funcionado:
print("\nCantidad de datos nulos:", dataset_2020.genero_usuario.isnull().sum(), "\n")

#### <u>2019:</u>

In [ ]:
dataset_2019.genero_usuario.value_counts()

A) Cambios en las categorías "NO INFORMADO" y en los valores erroneos:

Las 4 filas erroneas contienen nulos en las demás columnas, por lo que no se imputarán sino que se eliminarán:

In [ ]:
index_numeros = dataset_2019[dataset_2019["genero_usuario"] == "-58.4429517"].index
dataset_2019.drop(index_numeros, inplace = True)

Se reemplaza el valor "No informado" por la letra "N" utilizada en otro dataset:

In [ ]:
dataset_2019.loc[:,"genero_usuario"].replace({'NO INFORMADO':'N'}, inplace = True)

dataset_2019.genero_usuario.value_counts()

B) Cambios en columnas para hacer merge:

In [ ]:
print("Ya hay un", round(((len(dataset_2019)-dataset_2019.genero_usuario.isnull().sum())/len(dataset_2019)), 2)*100, "% de los datos válidos")

Como en este dataset ya existe la columna género con algunos datos, hay que renombrarla y así evitar que se dupliquen filas

In [ ]:
dataset_2019.rename(columns={"genero_usuario": "genero_usuario_viejo"}, inplace = True)

In [ ]:
dataframe_id_genero_2019 = dataframe_id_genero.loc[dataframe_id_genero.id_usuario.isin(dataset_2019.id_usuario), :]
dataframe_id_genero_2019

B) Aplicación del merge entre el nuevo dataframe y el dataset:

In [ ]:
dataset_2019 = pd.merge(dataset_2019, dataframe_id_genero_2019, how = "outer")
# Cálculo de resultados:
print("\nQuedaron:", dataset_2019.genero_usuario.isnull().sum(), "nulos restantes\nLa imputación anterior completó correctamente otro",
      (round(((len(dataset_2019)-dataset_2019.genero_usuario.isnull().sum())/len(dataset_2019)), 2)*100), "% de los datos faltantes, totalizando \n")

C) Unificación de ambas imputaciones

In [ ]:
dataset_2019["genero_usuario"] = dataset_2019["genero_usuario"].combine_first(dataset_2019["genero_usuario_viejo"])

In [ ]:
dataset_2019.drop("genero_usuario_viejo", axis = 1, inplace = True)
dataset_2019

In [ ]:
# Cálculo de resultados:
proporcion_validos = round((len(dataset_2019)-dataset_2019.genero_usuario.isnull().sum())/len(dataset_2019), 3)
print("\nQuedaron:", dataset_2019.genero_usuario.isnull().sum(), "nulos restantes al unir ambos resultados, es decir, un",
      proporcion_validos*100, "% de los datos son válidos\n")

D) Imputación de los valores restantes

Utilizar un 23% de los datos para imputar el 77% restante no es estadísticamente robusto, por lo que se utilizará un punto medio entre las proporciones del 23% válido de 2019 y las proporciones obtenidas al promediar 2018 y 2020:

In [ ]:
value_counts_2019 = round(dataset_2019.genero_usuario.value_counts(normalize = True),3)
value_counts_2019

In [ ]:
promedio_2018_2020 = round((dataset_2018.genero_usuario.value_counts(normalize = True) + dataset_2020.genero_usuario.value_counts(normalize = True)) /2, 3)
promedio_2018_2020

El valor N da NaN por no estar presente en el dataset 2018. Se utilizarán las otras dos proporciones para imputar:

In [ ]:
value_counts_promedio = round((dataset_2019.genero_usuario.value_counts(normalize = True) + promedio_2018_2020) /2, 3)
value_counts_promedio

In [ ]:
# Imputación:
dataset_2019.loc[dataset_2019["genero_usuario"].isnull(), "genero_usuario"] = np.random.choice(size=dataset_2019["genero_usuario"].isnull().sum(), a=["M", "F"], p=[value_counts_promedio[1], value_counts_promedio[0]])
# Se chequea que las proporciones se hayan mantenido similares:
print("Las proporciones deberían ser:\n")
print(round(value_counts_promedio * (1-proporcion_validos) + ((dataset_2019.genero_usuario.value_counts(normalize = True) + promedio_2018_2020) /2) * proporcion_validos, 3))
print("\n\ny dieron:\n")
print(round(dataset_2019["genero_usuario"].value_counts(normalize = True),3))

In [ ]:
# Se corrobora que haya funcionado:
print("\nCantidad de datos nulos:", dataset_2019.genero_usuario.isnull().sum(), "\n")

### <u>Limpieza N° 2 - Duración</u>

#### <u>2015 - 2019:</u>

In [ ]:
dataset_2015_2019 = pd.concat([dataset_2015, dataset_2016, dataset_2017, dataset_2018, dataset_2019])

In [ ]:
dataset_2015_2019 = dataset_2015_2019[dataset_2015_2019.duracion_recorrido.notnull()]

In [ ]:
duracion_recorrido = dataset_2015_2019.loc[:,"duracion_recorrido"]
duracion_split_2015_2019 = duracion_recorrido.str.split(":", expand = True)[1]
dataset_2015_2019 = pd.concat((dataset_2015_2019, duracion_split_2015_2019), axis = 1).rename(columns= {1 : "minutos"})
dataset_2015_2019.drop(["duracion_recorrido"], axis = 1, inplace = True)

In [ ]:
sum(dataset_2015_2019["minutos"].isnull())

In [ ]:
pbar = tqdm([2015, 2016, 2017, 2018, 2019])

for año in pbar:
    temp_dataset = dataset_2015_2019[dataset_2015_2019.año == año]
    temp_name = "dataset_" + str(año)
    globals()[f"{temp_name}"] = temp_dataset

#### <u>2020 - 2021:</u>

In [ ]:
dataset_2020_2021 = pd.concat([dataset_2020, dataset_2021])

In [ ]:
dataset_2020_2021["duracion_recorrido"] = round(dataset_2020_2021["duracion_recorrido"].astype("int") / 60).astype("int")
dataset_2020_2021.rename(columns= {"duracion_recorrido": "minutos"}, inplace = True)

In [ ]:
sum(dataset_2020_2021.minutos.isnull())

In [ ]:
pbar = tqdm([2020, 2021])

for año in pbar:
    temp_dataset = dataset_2020_2021[dataset_2020_2021.año == año]
    temp_name = "dataset_" + str(año)
    globals()[f"{temp_name}"] = temp_dataset

### <u>Limpieza N° 3 - Estación de origen y destino</u>

Se limpia tanto el id, el nombre, la latitud y la longitud de la estación pues son datos relacionados.

Utilizando los datasets de estaciones que provee el Gobierno de la Ciudad, disponibles en https://data.buenosaires.gob.ar/dataset/estaciones-bicicletas-publicas, se observa que la discrepancia en id de los datasets de bicicletas surge por la presencia de dos números que identifican estaciones: id y código.

Lamentablemente, desde el nuevo sistema (2019 en adelante) el identificador de estación cambió, por lo que el dato "id_estacion" del primer sistema es el código que aparece a veces al principio de "nombre_estacion". Es por eso que se lo tendrá que extraer para reemplazar el id_estacion. En los casos en los que el nombre no incluya el código, deberá ser buscado en otras filas o, de no existir, en la base de datos de estaciones.

In [ ]:
estaciones_nuevo = pd.read_csv("https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Gobierno%20Abierto%20(Python)/Datasets/nuevas-estaciones-bicicletas-publicas.csv", sep=",", encoding='utf-8')
estaciones_viejo = pd.read_csv("https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Gobierno%20Abierto%20(Python)/Datasets/estaciones_sistema_viejo.csv", sep=",", encoding='utf-8')

In [ ]:
estaciones_nuevo

In [ ]:
estaciones_viejo

#### <u>2021:</u>

In [ ]:
dataset_2021

In [ ]:
%%capture [--no-stderr]
dataset_2021.loc[dataset_2021.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2021.loc[dataset_2021.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2021.loc[dataset_2021.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2021.loc[dataset_2021.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2021.loc[dataset_2021.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2021.loc[dataset_2021.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2021.loc[dataset_2021.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2021.loc[dataset_2021.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2021.isnull().sum()

In [ ]:
dataset_2021[dataset_2021.id_estacion_destino.isnull()]

In [ ]:
# Debido a discrepancias entre id y nombre, sumado a la falta de información sobre latitud y longitud, se borrarán estas filas:
dataset_2021 = dataset_2021[~pd.isnull(dataset_2021.id_estacion_destino)]

In [ ]:
# Se eliminan unas filas con información dudosa (estación inexistente en la base de datos de estaciones y con un código alto)
dataset_2021 = dataset_2021.loc[dataset_2021.nombre_estacion_destino != "balboa definitivo",:]
dataset_2021

In [ ]:
# Se separa el código del nombre
# Origen:
col_nombre_origen = dataset_2021.loc[:,"nombre_estacion_origen"]
nombre_origen_2021 = col_nombre_origen.str.split("-", expand = True)[1].str.strip()
id_origen_2021 = col_nombre_origen.str.split("-", expand = True)[0].astype("int")
dataset_2021 = pd.concat((dataset_2021, id_origen_2021, nombre_origen_2021), axis = 1).rename(columns= {1 : "nombre_origen", 0 : "codigo_origen"})

# Destino:
col_nombre_destino = dataset_2021.loc[:,"nombre_estacion_destino"].str.strip()
nombre_destino_2021 = col_nombre_destino.str.split("-", expand = True)[1]
id_destino_2021 = col_nombre_destino.str.split("-", expand = True)[0].astype("int")
dataset_2021 = pd.concat((dataset_2021, id_destino_2021, nombre_destino_2021), axis = 1).rename(columns= {1 : "nombre_destino", 0 : "codigo_destino"})

# Drop de columnas viejas:
dataset_2021.drop(["nombre_estacion_origen", "id_estacion_origen", "id_estacion_destino", "nombre_estacion_destino"], axis = 1, inplace = True)
dataset_2021

In [ ]:
drop_duplicados = dataset_2021.drop_duplicates(subset = ["codigo_origen", "nombre_origen"])
duplicados_first = drop_duplicados.duplicated(subset = ["codigo_origen"])
drop_duplicados[duplicados_first]

Los nulos de lat y long en todas las filas se resolverán luego al emprolijar el resto del dataset

#### <u>2020:</u>

In [ ]:
dataset_2020

In [ ]:
%%capture [--no-stderr]
dataset_2020.loc[dataset_2020.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2020.loc[dataset_2020.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2020.loc[dataset_2020.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2020.loc[dataset_2020.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2020.loc[dataset_2020.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2020.loc[dataset_2020.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2020.loc[dataset_2020.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2020.loc[dataset_2020.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2020.isnull().sum()

In [ ]:
# Se separa el código del nombre
# Origen:
col_nombre_origen = dataset_2020.loc[:,"nombre_estacion_origen"]
nombre_origen_2020 = col_nombre_origen.str.split("-", expand = True)[1].str.strip()
id_origen_2020 = col_nombre_origen.str.split("-", expand = True)[0].astype("int")
dataset_2020 = pd.concat((dataset_2020, id_origen_2020, nombre_origen_2020), axis = 1).rename(columns= {1 : "nombre_origen", 0 : "codigo_origen"})

# Destino:
col_nombre_destino = dataset_2020.loc[:,"nombre_estacion_destino"]
nombre_destino_2020 = col_nombre_destino.str.split("-", expand = True)[1].str.strip()
id_destino_2020 = col_nombre_destino.str.split("-", expand = True)[0].astype("int")
dataset_2020 = pd.concat((dataset_2020, id_destino_2020, nombre_destino_2020), axis = 1).rename(columns= {1 : "nombre_destino", 0 : "codigo_destino"})

# Drop de columnas viejas:
dataset_2020.drop(["nombre_estacion_origen", "id_estacion_origen", "id_estacion_destino", "nombre_estacion_destino"], axis = 1, inplace = True)
dataset_2020

In [ ]:
dataset_2020.isnull().sum()

In [ ]:
drop_duplicados = dataset_2020.drop_duplicates(subset = ["codigo_origen", "nombre_origen"])
drop_duplicados

In [ ]:
duplicados_first = drop_duplicados.duplicated(subset = ["codigo_origen"])
duplicados_last = drop_duplicados.duplicated(subset = ["codigo_origen"], keep = "last")
drop_duplicados[duplicados_first | duplicados_last]

In [ ]:
drop_duplicados_destino = dataset_2020.drop_duplicates(subset = ["codigo_destino", "nombre_destino"])
drop_duplicados_destino

In [ ]:
duplicados_first = drop_duplicados_destino.duplicated(subset = ["codigo_destino"])
duplicados_last = drop_duplicados_destino.duplicated(subset = ["codigo_destino"], keep = "last")
drop_duplicados_destino[duplicados_first | duplicados_last]

In [ ]:
estaciones_nuevo.loc[estaciones_nuevo.codigo == 150, :]

In [ ]:
estaciones_nuevo.loc[estaciones_nuevo.WKT.str.contains('-58.355744'),:]

In [ ]:
# Vera Peñaloza no ofrece resultados. Se cambiará el nombre por Rodrigo Bueno:
dataset_2020.replace({"vera penaloza" : "rodrigo bueno"}, regex=True, inplace = True)

In [ ]:
drop_duplicados_destino = dataset_2020.drop_duplicates(subset = ["codigo_destino", "nombre_destino"])
duplicados_first = drop_duplicados_destino.duplicated(subset = ["codigo_destino"])
duplicados_last = drop_duplicados_destino.duplicated(subset = ["codigo_destino"], keep = "last")
drop_duplicados_destino[duplicados_first | duplicados_last]

#### <u>2019:</u>

In [ ]:
%%capture [--no-stderr]
dataset_2019.loc[dataset_2019.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2019.loc[dataset_2019.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2019.loc[dataset_2019.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2019.loc[dataset_2019.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2019

In [ ]:
# Hay un caso con nombre diferenciado por "&" vs "y":
fitz_roy = dataset_2019.loc[dataset_2019.nombre_estacion_origen.str.contains('fitz'),"nombre_estacion_origen"]
fitz_roy

In [ ]:
%%capture [--no-stderr]
dataset_2019.replace({"fitz roy y gorriti" : "fitz roy & gorriti"}, regex=True, inplace = True)

In [ ]:
# Se separa el código del nombre. Lamentablemente no todas las filas lo tienen, por lo que se filtra las que tienen "-" y, en los casos negativos se mantendrá el id original

# Origen:
col_nombre_origen = dataset_2019.loc[dataset_2019.nombre_estacion_destino.str.contains('-'),"nombre_estacion_origen"]
nombre_origen_2019 = col_nombre_origen.str.split("-", expand = True)[1].str.strip()
id_origen_2019 = col_nombre_origen.str.split("-", expand = True)[0].astype("int")
dataset_2019 = pd.concat((dataset_2019, id_origen_2019, nombre_origen_2019), axis = 1).rename(columns= {1 : "nombre_origen", 0 : "codigo_origen"})

# Destino:
col_nombre_destino = dataset_2019.loc[dataset_2019.nombre_estacion_destino.str.contains('-'),"nombre_estacion_destino"]
nombre_destino_2019 = col_nombre_destino.str.split("-", expand = True)[1].str.strip()
id_destino_2019 = col_nombre_destino.str.split("-", expand = True)[0].astype("int")
dataset_2019 = pd.concat((dataset_2019, id_destino_2019, nombre_destino_2019), axis = 1).rename(columns= {1 : "nombre_destino", 0 : "codigo_destino"})

In [ ]:
dataset_2019

In [ ]:
dataset_2019.isnull().sum()

In [ ]:
dataframe_codigo_origen_2019 = dataset_2019.loc[:,["id_estacion_origen", "codigo_origen","nombre_origen"]]
dataframe_codigo_origen_2019.dropna(subset = ["codigo_origen"], inplace = True)
dataframe_codigo_origen_2019.drop_duplicates(subset = ["codigo_origen"], inplace = True)
dataframe_codigo_origen_2019.rename(columns = {"codigo_origen" : "codigo_origen_merge", "nombre_origen" : "nombre_origen_merge"}, inplace = True)
dataframe_codigo_origen_2019

In [ ]:
dataset_2019 = dataset_2019.merge(dataframe_codigo_origen_2019, on='id_estacion_origen', how='left')

In [ ]:
dataframe_codigo_destino_2019 = dataset_2019.loc[:,["id_estacion_destino", "codigo_destino","nombre_destino"]]
dataframe_codigo_destino_2019.dropna(subset = ["codigo_destino"], inplace = True)
dataframe_codigo_destino_2019.drop_duplicates(subset = ["codigo_destino"], inplace = True)
dataframe_codigo_destino_2019.rename(columns = {"codigo_destino" : "codigo_destino_merge", "nombre_destino" : "nombre_destino_merge"}, inplace = True)
dataframe_codigo_destino_2019

In [ ]:
dataset_2019 = dataset_2019.merge(dataframe_codigo_destino_2019, on='id_estacion_destino', how='left')

In [ ]:
dataset_2019

In [ ]:
dataset_2019.isnull().sum()

In [ ]:
drop_duplicados_origen = dataset_2019.drop_duplicates(subset = ["codigo_origen_merge", "nombre_origen_merge"])
duplicados_first = drop_duplicados_origen.duplicated(subset = ["codigo_origen_merge"])
duplicados_last = drop_duplicados_origen.duplicated(subset = ["codigo_origen_merge"], keep = "last")
drop_duplicados_origen[duplicados_first | duplicados_last]

In [ ]:
drop_duplicados_destino = dataset_2019.drop_duplicates(subset = ["codigo_destino_merge", "nombre_destino_merge"])
duplicados_first = drop_duplicados_destino.duplicated(subset = ["codigo_destino_merge"])
duplicados_last = drop_duplicados_destino.duplicated(subset = ["codigo_destino_merge"], keep = "last")
drop_duplicados_destino[duplicados_first | duplicados_last]

In [ ]:
dataset_2019.dropna(subset = ["codigo_origen_merge", "nombre_origen_merge", "codigo_destino_merge", "nombre_destino_merge"], inplace = True)
dataset_2019.drop(["nombre_estacion_origen", "id_estacion_origen", "nombre_estacion_destino", "id_estacion_destino", "codigo_origen", "nombre_origen", "codigo_destino", "nombre_destino"], axis = 1, inplace = True)
dataset_2019.rename(columns = {"codigo_origen_merge" : "codigo_origen", "nombre_origen_merge" : "nombre_origen", "codigo_destino_merge" : "codigo_destino", "nombre_destino_merge" : "nombre_destino"}, inplace = True)
dataset_2019

In [ ]:
dataset_2019.isnull().sum()

In [ ]:
dataset_2019[dataset_2019.lat_estacion_destino.isnull()]

In [ ]:
eco_lat = dataset_2019.loc[(dataset_2019.nombre_destino == "ecoparque") & (dataset_2019.lat_estacion_destino.notnull()), "lat_estacion_destino"].iloc[0]
eco_long = dataset_2019.loc[(dataset_2019.nombre_destino == "ecoparque") & (dataset_2019.long_estacion_destino.notnull()), "long_estacion_destino"].iloc[0]

dataset_2019.fillna(value = {'lat_estacion_destino': eco_lat, 'long_estacion_destino': eco_long}, inplace = True)
dataset_2019.isnull().sum()

### <u>2018 - 2015:</u>

En estos años "id_estacion" es el código de los años 2019 a 2021, por lo que la limpieza anterior no es necesaria

- A) 2018

In [ ]:
%%capture [--no-stderr]
dataset_2018.loc[dataset_2018.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2018.loc[dataset_2018.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2018.loc[dataset_2018.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2018.loc[dataset_2018.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2018.loc[dataset_2018.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2018.loc[dataset_2018.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2018.loc[dataset_2018.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2018.loc[dataset_2018.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2018.isnull().sum()

In [ ]:
dataset_2018[dataset_2018.id_estacion_destino.isnull()].nombre_estacion_destino.value_counts()

In [ ]:
dataset_2018[dataset_2018.id_estacion_origen.isnull()].nombre_estacion_origen.value_counts()

In [ ]:
%%capture [--no-stderr]
# Ecoparque:
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "ecoparque", "id_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "ecoparque", "codigo_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "ecoparque", "id_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "ecoparque", "codigo_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "ecoparque", "lat_estacion_origen"] = eco_lat
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "ecoparque", "long_estacion_origen"] = eco_long
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "ecoparque", "lat_estacion_destino"] = eco_lat
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "ecoparque", "long_estacion_destino"] = eco_long

# Fitz Roy y Gorriti:
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "fitz roy y gorriti", "id_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "codigo_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "fitz roy y gorriti", "id_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "codigo_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "fitz roy y gorriti", "lat_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "lat_estacion_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_origen == "fitz roy y gorriti", "long_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "long_estacion_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "fitz roy y gorriti", "lat_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "lat_estacion_destino"].iloc[0]
dataset_2018.loc[dataset_2018.nombre_estacion_destino == "fitz roy y gorriti", "long_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "long_estacion_destino"].iloc[0]

In [ ]:
%%capture [--no-stderr]
dataset_2018.rename(columns = {"id_estacion_origen" : "codigo_origen", "id_estacion_destino" : "codigo_destino", "nombre_estacion_origen" : "nombre_origen", "nombre_estacion_destino" : "nombre_destino"}, inplace = True)

In [ ]:
dataset_2018.isnull().sum()

- B) 2017

In [ ]:
%%capture [--no-stderr]
dataset_2017.loc[dataset_2017.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2017.loc[dataset_2017.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2017.loc[dataset_2017.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2017.loc[dataset_2017.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2017.loc[dataset_2017.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2017.loc[dataset_2017.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2017.loc[dataset_2017.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2017.loc[dataset_2017.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2017.isnull().sum()

In [ ]:
dataset_2017[dataset_2017.id_estacion_destino.isnull()].nombre_estacion_destino.value_counts()

In [ ]:
%%capture [--no-stderr]
# Se corrige el error de tipeo en "gorrtiti
dataset_2017.replace({"fitz roy y gorrtiti" : "fitz roy & gorriti"}, regex=True, inplace = True)                      

In [ ]:
%%capture [--no-stderr]
# Ecoparque:
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "f. j. santamaria de oro", "id_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "codigo_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "f. j. santamaria de oro", "id_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "codigo_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "f. j. santamaria de oro", "lat_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "lat_estacion_origen"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "f. j. santamaria de oro", "long_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "long_estacion_origen"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "f. j. santamaria de oro", "lat_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "lat_estacion_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "f. j. santamaria de oro", "long_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "f.j.santamaria de oro", "long_estacion_destino"].iloc[0]

# Fitz Roy y Gorriti:
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "fitz roy & gorriti", "id_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "codigo_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "fitz roy & gorriti", "id_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "codigo_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "fitz roy & gorriti", "lat_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "lat_estacion_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_origen == "fitz roy & gorriti", "long_estacion_origen"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "long_estacion_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "fitz roy & gorriti", "lat_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "lat_estacion_destino"].iloc[0]
dataset_2017.loc[dataset_2017.nombre_estacion_destino == "fitz roy & gorriti", "long_estacion_destino"] = dataset_2019.loc[dataset_2019.nombre_destino == "fitz roy & gorriti", "long_estacion_destino"].iloc[0]

In [ ]:
%%capture [--no-stderr]
dataset_2017.rename(columns = {"id_estacion_origen" : "codigo_origen", "id_estacion_destino" : "codigo_destino", "nombre_estacion_origen" : "nombre_origen", "nombre_estacion_destino" : "nombre_destino"}, inplace = True)

In [ ]:
dataset_2017.isnull().sum()

- C) 2016

In [ ]:
%%capture [--no-stderr]
dataset_2016.loc[dataset_2016.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2016.loc[dataset_2016.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2016.loc[dataset_2016.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2016.loc[dataset_2016.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2016.loc[dataset_2016.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2016.loc[dataset_2016.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2016.loc[dataset_2016.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2016.loc[dataset_2016.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2016.isnull().sum()

In [ ]:
%%capture [--no-stderr]
dataset_2016.rename(columns = {"id_estacion_origen" : "codigo_origen", "id_estacion_destino" : "codigo_destino", "nombre_estacion_origen" : "nombre_origen", "nombre_estacion_destino" : "nombre_destino"}, inplace = True)

- D) 2015

In [ ]:
%%capture [--no-stderr]
dataset_2015.loc[dataset_2015.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2015.loc[dataset_2015.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].apply(lambda x : unidecode.unidecode(x))
dataset_2015.loc[dataset_2015.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"] = dataset_2015.loc[dataset_2015.nombre_estacion_origen.isnull() == False, "nombre_estacion_origen"].str.lower()

dataset_2015.loc[dataset_2015.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2015.loc[dataset_2015.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].apply(lambda x : unidecode.unidecode(x))
dataset_2015.loc[dataset_2015.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"] = dataset_2015.loc[dataset_2015.nombre_estacion_destino.isnull() == False, "nombre_estacion_destino"].str.lower()

In [ ]:
dataset_2015.isnull().sum()

In [ ]:
dataset_2015[dataset_2015.lat_estacion_destino.isnull()].nombre_estacion_destino.value_counts()

In [ ]:
estaciones_nuevo.loc[estaciones_nuevo.nombre.str.contains("stero"), :]
estaciones_viejo.loc[estaciones_viejo.nombre_estacion.str.contains("stero"), :]

estaciones_nuevo.loc[estaciones_nuevo.nombre.str.contains("chaba"), :]
estaciones_viejo.loc[estaciones_viejo.nombre_estacion.str.contains("chaba"), :]

In [ ]:
%%capture [--no-stderr]
# Como las dos estaciones con nulos no se encuentran en los otros datasets o en el de estaciones del sistema nuevo y el dataset del sistema viejo contiene ambos pero sin la información de latitud y longitud, se procederá a eliminarlos.
dataset_2015.dropna(subset = ["long_estacion_origen", "lat_estacion_origen", "long_estacion_destino", "lat_estacion_destino"], inplace = True)

In [ ]:
%%capture [--no-stderr]
dataset_2015.rename(columns = {"id_estacion_origen" : "codigo_origen", "id_estacion_destino" : "codigo_destino", "nombre_estacion_origen" : "nombre_origen", "nombre_estacion_destino" : "nombre_destino"}, inplace = True)

In [ ]:
dataset_2015.isnull().sum()

# 3) Unificación, limpieza final y exportación del dataset

### <u>A) Concatenación parcial + imputación de longitudes y latitudes de 2021:</u>

In [ ]:
# No se concatena el año 2021 pues se le tiene que imputar las latitudes y longitudes
dataset = pd.concat([dataset_2015, dataset_2016, dataset_2017, dataset_2018, dataset_2019, dataset_2020])
dataset

In [ ]:
%%capture [--no-stderr]
# Se crea un df con latitudes y longitudes por código de los años 2015 a 2020

dataframe_lat_long_origen = dataset.loc[dataset.año != "2021",["codigo_origen", "lat_estacion_origen","long_estacion_origen"]]
dataframe_lat_long_origen.drop_duplicates(subset = ["codigo_origen"], inplace = True)
dataframe_lat_long_origen.rename(columns = {"lat_estacion_origen" : "lat_estacion_origen_merge", "long_estacion_origen" : "long_estacion_origen_merge"}, inplace = True)
dataframe_lat_long_origen.dropna(subset = ["lat_estacion_origen_merge", "long_estacion_origen_merge"], inplace = True)

dataframe_lat_long_destino = dataset.loc[dataset.año != "2021",["codigo_destino", "lat_estacion_destino","long_estacion_destino"]]
dataframe_lat_long_destino.drop_duplicates(subset = ["codigo_destino"], inplace = True)
dataframe_lat_long_destino.rename(columns = {"lat_estacion_destino" : "lat_estacion_destino_merge", "long_estacion_destino" : "long_estacion_destino_merge"}, inplace = True)
dataframe_lat_long_destino.dropna(subset = ["lat_estacion_destino_merge", "long_estacion_destino_merge"], inplace = True)

In [ ]:
dataframe_lat_long_destino

In [ ]:
# Se hace un merge para adjudicar a cada estación de origen/destino su latitud y longitud según su código

dataset_2021 = dataset_2021.merge(dataframe_lat_long_origen, on='codigo_origen', how='left')
dataset_2021 = dataset_2021.merge(dataframe_lat_long_destino, on='codigo_destino', how='left')
dataset_2021

In [ ]:
# Se hace un drop de las columnas con nulos y se reemplazan los nombres de las columnas nuevas

dataset_2021.drop(["lat_estacion_origen", "long_estacion_origen", "lat_estacion_destino", "long_estacion_destino"], axis = 1, inplace = True)
dataset_2021.rename(columns = {"lat_estacion_origen_merge" : "lat_estacion_origen", "long_estacion_origen_merge" : "long_estacion_origen", "lat_estacion_destino_merge" : "lat_estacion_destino", "long_estacion_destino_merge" : "long_estacion_destino"}, inplace = True)
print("\nQuedaron:", dataset_2021.lat_estacion_origen.isnull().sum(), "nulos restantes\nLa imputación anterior completó correctamente el",
      (round(((len(dataset_2021)-dataset_2021.lat_estacion_origen.isnull().sum())/len(dataset_2021)), 2)*100), "% de los datos faltantes\n")

In [ ]:
dataset_2021.dropna(subset = ["lat_estacion_origen", "long_estacion_origen", "lat_estacion_destino", "long_estacion_destino"], inplace = True)

### <u>B) Concatenación final y corrección de algunos textos:</u>

In [ ]:
dataset = pd.concat([dataset, dataset_2021])
dataset

In [ ]:
dataset.dtypes

In [ ]:
%%capture [--no-stderr]
dataset.codigo_origen = dataset.codigo_origen.astype("int")
dataset.codigo_destino = dataset.codigo_destino.astype("int")
dataset.minutos = dataset.minutos.astype("int")
dataset.loc[dataset.id_usuario.isnull() == False, "id_usuario"] = dataset.loc[dataset.id_usuario.isnull() == False, "id_usuario"].astype("int")

In [ ]:
%%capture [--no-stderr]
# Se corrige el espaciado en "f.j.santamaria" y se unifica "&" con "y"
dataset.replace({"f.j.santamaria de oro" : "f. j. santamaria de oro", " & " : " y "}, regex=True, inplace = True)   

In [ ]:
dataset.isnull().sum()

Lamentablemente los id_usuario nulos son imposibles de imputar, pero se deja esa columna pues se puede utilizar los años que tienen ese dato (los cuales no presentan nulos)

### <u>C) Eliminación de outliers:</u>

La duración de los recorridos presenta algunos números errones:

In [ ]:
print((dataset["minutos"]).max())

In [ ]:
q75,q25 = np.percentile(dataset["minutos"],[75,25])
iqr = q75-q25
max_limit = q75+(1.5*iqr)
min_limit = q25-(1.5*iqr)
print("límite superior:", max_limit, "\nlímite inferior:", min_limit)

In [ ]:
# Como no se desea quitar recorridos dentro de los 60 minutos permitidos, se definirá el límite superior como 60 minutos.
# Como carece de sentido utilizar un límite inferior negativo, se le asignará el valor de 0 minutos
max_limit = 60
min_limit = 0

In [ ]:
cant_valores = len(dataset)
dataset = dataset.loc[(dataset["minutos"] < max_limit) & (dataset["minutos"]  > min_limit),:]

print("Se borraron", cant_valores - len(dataset), "outliers, quedando un total de", len(dataset), "valores válidos entre 0 y 60 minutos")

### <u>D) Exportación de dataset:</u>

In [ ]:
dataset.to_csv("dataset.csv", index = False)